In [252]:
import SimFunctions
import SimClasses
import SimRNG
import simpy
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import probplot, kstest
import pickle
from scipy.stats.distributions import norm, expon, erlang, lognorm, weibull_min, gamma
import statsmodels.api as sm
warnings.filterwarnings("ignore")
# fix random number seed
np.random.seed(1)


# Classes

In [253]:
class Station:
    def __init__(self, station_id, level, capacity):
        self.id = station_id
        self.level = level
        self.capacity = capacity
        self.bikes = {}

        for i in range(level):
            bike_id = f"{station_id}-{i+1}"  # create unique bike ID
            self.bikes[bike_id] = True  # mark bike as available

    def rent_bike(self):
        # Request a bike from the station
        for bike_id, available in self.bikes.items():
            if available:
                self.bikes[bike_id] = False  # mark bike as rented
                self.level -= 1
                return bike_id

        return None

    def return_bike(self, bike_id):
        # Return a bike to the station
        if bike_id in self.bikes and not self.bikes[bike_id]:
            self.bikes[bike_id] = True  # mark bike as available
            self.level += 1
            return True

        return False

# Initialize the stations and create a dictionary mapping station IDs to Station instances
Stations = [Station(station_id=1, level=7, capacity=10), Station(station_id=2, level=15, capacity=20)]
StationDict = {station.id: station for station in Stations}

# Functions

In [254]:
ZSimRNG = SimRNG.InitializeRNSeed()
Calendar = SimClasses.EventCalendar()
TheQueues = []
TheCTStats = []
TheDTStats = []
TheResources = []

In [ ]:
# When a customer arrives:
customer_id = SimFunctions.NextCustomerID()
SimFunctions.Schedule(Calendar, f"Customer_Departure_{customer_id}", SimRNG.Expon(MeanTR, 2))
SimFunctions.Schedule(Calendar, f"Bike_Arrival_{customer_id}", SimRNG.Expon(MeanST, 3))

# When a customer departs:
SimFunctions.Schedule(Calendar, f"Retrial_{customer_id}", SimRNG.Expon(MeanTBA, 4))

# When a bike arrives:
SimFunctions.Schedule(Calendar, f"Customer_Arrival_{customer_id}", SimRNG.Expon(MeanTBA, 5))


In [310]:
class Customer:
    def __init__(self, customer_id, start_station, bike=None):
        self.customer_id = customer_id
        self.start_station = start_station
        self.bike = bike

    def rent_bike(self):
        station = StationDict[self.start_station]
        self.bike = station.rent_bike()
        if self.bike is None:
            # Bike not available, retry later
            print()
            SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(1, 5))
        else:
            # Bike rented successfully, schedule departure
            print("(CUSTOMER ARRIVAL) Customer ID: {} | Station ID: {} | Bike ID {}".format(self.customer_id, self.start_station, self.bike))
            SimFunctions.Schedule(Calendar, "Departure",2*SimRNG.Lognormal(0.1, np.sqrt(0.066), 6))

    def return_bike(self, end_station):
        destination_station = StationDict[end_station]
        destination_station.return_bike(self.bike)

        # if destination_station.level < destination_station.capacity:
        #     destination_station.return_bike(self.bike)
        #     print("(BIKE ARRIVAL) Customer ID: {} | Destination Station ID: {} | Bike ID: {}".format(self.customer_id, end_station, self.bike))
        # else:
        #     # Destination station is full, retry later
        #     print("FULL -- Customer ID: {} | Destination Station ID: {} -- FULL".format(self.customer_id, end_station))
        #     SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))


def Customer_Arrival(station_id, empty_error, CustomerList):
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(0.5, 1))
    station = StationDict[station_id]
    customer_id = NextCustomerID()
    customer = Customer(customer_id, station_id)
    CustomerList.append(customer)

    customer.start_station = station_id
    
    # Station is empty
    if station.level <= 0:
        print(f"    EMPRTY -- Customer Arrives at STATION {station.id} -- EMPTY")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
        empty_error += 1
    else:
        customer.rent_bike()

    return Customer_Arrival_Rate(station_id), empty_error


def Departure(station_id):
    station = StationDict[station_id]
    print("(DEPARTING) Station ID: {} | Level {}".format(station.id, station.level, station.capacity))
    print("-------------------------------------------------------------------------")

    end_id = Destination(station)
    SimFunctions.Schedule(Calendar, "Bike_Arrival", 2*SimRNG.Lognormal(0.1, np.sqrt(0.066), 4))

    return end_id


def Destination(station_id):
    #station = StationDict[station_id]
    U = SimRNG.Uniform(0, 1, 3)
    end_id = 2 if U > 1/2 else 1

    end_station = StationDict[end_id]
    print("(DESTINATION) Station ID: {} | Level: {}".format(end_station.id, end_station.level))
    print("-------------------------------------------------------------------------")
    return end_id


def Bike_Arrival(end_id, full_error, CustomerList):
    destination_station = StationDict[end_id]
    #bike = destination_station
    for customer in CustomerList:
        if customer.start_station == end_id and customer.bike is not None:
            print("(BIKE ARRIVAL) Customer ID: {} | Bike ID: {} | From: {} | To: {}".format(customer.customer_id, customer.bike, customer.start_station, end_id))
            print("-------------------------------------------------------------------------")
            customer.return_bike(end_id)
            CustomerList.remove(customer)
            return full_error
    #print(f"    BIKE NOT FOUND -- Bike Arrives at STATION {destination_station.id}")
    if destination_station.level >= destination_station.capacity:
        print(f"    FULL -- STATION {destination_station.id} FULL")
        full_error += 1
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
    return full_error

    
    # Station full or bike not found, schedule a retrial event
    print(f"    FULL -- Bike Arrives at STATION {destination_station.id} -- FULL")
    SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
    return full_error + 1

def Customer_Arrival_Rate(station_id):
    U = SimRNG.Uniform(0, 1, 3)
    station_id = 2 if U > 1/2 else 1
    return station_id

def Retrial(end_id, Full_Error, bike):
    print("Retrial")
    destination_station = StationDict[end_id]
    # Destination Station is not full
    if destination_station.level < destination_station.capacity:
        destination_station.return_bike(bike)
        print("     Bike has succesfully retried and returned bike to Station",
              destination_station.id)
    else:
        # STATION FULL
        print(f"    STATION {destination_station.id} FULL")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
        Full_Error += 1
    return Full_Error


def NextCustomerID():
    if not hasattr(NextCustomerID, "counter"):
        NextCustomerID.counter = 0
    NextCustomerID.counter += 1
    return NextCustomerID.counter

# Simulation

In [314]:
ZSimRNG = SimRNG.InitializeRNSeed()

Calendar = SimClasses.EventCalendar()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Bikes = SimClasses.Resource()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

# Stations = [7, 15]
# Station_Capacity = [10, 20]

Station_1_Resource = []
Station_2_Resource = []
CustomerList = []

MeanTBA = 0.5
MeanTR = 0.5
MeanST = 0.5

Phases = 3
RunLength = 50
WarmUp = 5
end_id = 1

for reps in range(0, 1, 1):
    Full_Error = 0
    Empty_Error = 0 
    
    inital_station_id = np.random.random_integers(1, 3)
    station_id = inital_station_id
    
    # Initialize the stations and create a dictionary mapping station IDs to Station instances
    Stations = [Station(station_id=1, level=15, capacity=20), Station(station_id=2, level=35, capacity=40), Station(station_id=3, level=24, capacity=25)]
    StationDict = {station.id: station for station in Stations}

    SimFunctions.SimFunctionsInit(Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(MeanTBA, 1))

    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Customer_Arrival":
        Customer_Arrival(inital_station_id, Empty_Error, CustomerList)


    for hours in [8, 8.3]:#, 9, 9.3, 10, 10.3, 11, 11.3, 12]:  # 30 min intervals
        print(f"Clock Hour | {hours}")
        for minutes in [i for i in range(1, 31)]:  # 30 min intervals
            print("Minute | {}".format(minutes))
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            
            if NextEvent.EventType == "Customer_Arrival":
                station_id, Empty_Error = Customer_Arrival(station_id, Empty_Error, CustomerList)

            elif NextEvent.EventType == "Departure":
                end_id = Departure(station_id)
            
            elif NextEvent.EventType == "Bike_Arrival":
                Full_Error = Bike_Arrival(end_id, Full_Error, CustomerList)
            
            elif NextEvent.EventType == "Retrial":
                Full_Error = Retrial(end_id, Full_Error, bike)
            

               
    print(StationDict[1].bikes)
    print(StationDict[2].bikes)
    print(StationDict[3].bikes)
    print("Num of Full Errors:", Full_Error)
    print("Num of Empty Errors:", Empty_Error)

(CUSTOMER ARRIVAL) Customer ID: 64 | Station ID: 3 | Bike ID 3-1
Clock Hour | 8
Minute | 1
(DEPARTING) Station ID: 3 | Level 23
-------------------------------------------------------------------------
(DESTINATION) Station ID: 1 | Level: 15
-------------------------------------------------------------------------
Minute | 2
Minute | 3
(CUSTOMER ARRIVAL) Customer ID: 65 | Station ID: 3 | Bike ID 3-2
Minute | 4
(DEPARTING) Station ID: 1 | Level 15
-------------------------------------------------------------------------
(DESTINATION) Station ID: 1 | Level: 15
-------------------------------------------------------------------------
Minute | 5
Minute | 6
(CUSTOMER ARRIVAL) Customer ID: 66 | Station ID: 1 | Bike ID 1-1
Minute | 7
(DEPARTING) Station ID: 1 | Level 14
-------------------------------------------------------------------------
(DESTINATION) Station ID: 1 | Level: 14
-------------------------------------------------------------------------
Minute | 8
(BIKE ARRIVAL) Customer ID:

In [300]:
def Departure(station_id):
    station = StationDict[station_id]
    print("(DEPARTING) Station ID: {} | Level {}".format(
        station.id, station.level, station.capacity))
    print("-------------------------------------------------------------------------")

    end_id = Destination(station)
    SimFunctions.Schedule(Calendar, "Bike_Arrival", 2 *
                          SimRNG.Lognormal(0.1, np.sqrt(0.066), 4))

    # Rent a bike from the station
    station.rent_bike()
    return end_id


def Destination(station_id):
    # print("Destination")
    #station = StationDict[station_id]

    U = SimRNG.Uniform(0, 1, 3)
    end_id = 2 if U > 1/2 else 1

    end_station = StationDict[end_id]
    print("(DESTINATION) Station ID: {} | Level: {}".format(
        end_station.id, end_station.level))
    print("-------------------------------------------------------------------------")
    return end_id


def Customer_Arrival_Rate(station_id):
    U = SimRNG.Uniform(0, 1, 3)
    station_id = 2 if U > 1/2 else 1
    return station_id


def Customer_Arrival(station_id, Empty_Error, bike):
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(0.5, 1))
    station = StationDict[station_id]

    new_station_id = Customer_Arrival_Rate(station_id)

    # Station is empty
    if station.level <= 0:
        print(f"    Customer Arrives at STATION {station.id} -- EMPTY")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
        Empty_Error += 1
    else:
        bike = station.rent_bike()
        if bike is None:
            print(f"    Customer Arrives at STATION {station.id} -- EMPTY")
            SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(1, 5))
            Empty_Error += 1
        else:
            print("(CUSTOMER ARRIVAL) Station ID: {} | Level: {} | Capacity: {} | Bike ID {}".format(
                station.id, station.level, station.capacity, bike))
            print(
                "-------------------------------------------------------------------------")
            SimFunctions.Schedule(Calendar, "Departure",
                                  2*SimRNG.Lognormal(0.1, np.sqrt(0.066), 6))

    return new_station_id, Empty_Error, bike


def Bike_Arrival(end_id, Full_Error, bike):
    destination_station = StationDict[end_id]
    # Destination Station is not full
    if destination_station.level < destination_station.capacity:
        destination_station.return_bike(bike)
        print("(BIKE ARRIVAL) Station ID: {} | Level: {} | Bike ID: {}".format(
            destination_station.id, destination_station.level, bike))
        print("-------------------------------------------------------------------------")
    else:
        # STATION FULL
        print(f"    Bike Arrives at STATION {destination_station.id} -- FULL")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
        Full_Error += 1
    return Full_Error


def Retrial(end_id, Full_Error, bike):
    print("Retrial")
    destination_station = StationDict[end_id]
    # Destination Station is not full
    if destination_station.level < destination_station.capacity:
        destination_station.return_bike(bike)
        print("     Bike has succesfully retried and returned bike to Station",
              destination_station.id)
    else:
        # STATION FULL
        print(f"    STATION {destination_station.id} FULL")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
        Full_Error += 1
    return Full_Error


def NextCustomerID():
    if not hasattr(NextCustomerID, "counter"):
        NextCustomerID.counter = 0
    NextCustomerID.counter += 1
    return NextCustomerID.counter


In [301]:
ZSimRNG = SimRNG.InitializeRNSeed()

Calendar = SimClasses.EventCalendar()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Bikes = SimClasses.Resource()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

# Stations = [7, 15]
# Station_Capacity = [10, 20]

Station_1_Resource = []
Station_2_Resource = []

MeanTBA = 0.5
MeanTR = 0.5
MeanST = 0.5

Phases = 3
RunLength = 50
WarmUp = 5
end_id = 1

for reps in range(0, 1, 1):
    Full_Error = 0
    Empty_Error = 0

    inital_station_id = np.random.random_integers(1, 2)
    station_id = inital_station_id

    # Initialize the stations and create a dictionary mapping station IDs to Station instances
    Stations = [Station(station_id=1, level=15, capacity=20),
                Station(station_id=2, level=35, capacity=40)]
    StationDict = {station.id: station for station in Stations}

    SimFunctions.SimFunctionsInit(
        Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
    SimFunctions.Schedule(Calendar, "Customer_Arrival",
                          SimRNG.Expon(MeanTBA, 1))

    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Customer_Arrival":
        Customer_Arrival(inital_station_id, Empty_Error, 0)

    for hours in [8, 8.3]:  # , 9, 9.3, 10, 10.3, 11, 11.3, 12]:  # 30 min intervals
        print(f"Clock Hour | {hours}")
        for minutes in [i for i in range(1, 31)]:  # 30 min intervals
            print("Minute | {}".format(minutes))
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime

            if NextEvent.EventType == "Customer_Arrival":
                station_id, Empty_Error, bike = Customer_Arrival(
                    station_id, Empty_Error, bike)

            elif NextEvent.EventType == "Departure":
                end_id = Departure(station_id)

            elif NextEvent.EventType == "Bike_Arrival":
                Full_Error = Bike_Arrival(end_id, Full_Error, bike)

            elif NextEvent.EventType == "Retrial":
                Full_Error = Retrial(end_id, Full_Error, bike)

    print(StationDict[1].bikes)
    print(StationDict[2].bikes)
    print("Num of Full Errors:", Full_Error)
    print("Num of Empty Errors:", Empty_Error)


(CUSTOMER ARRIVAL) Station ID: 1 | Level: 14 | Capacity: 20 | Bike ID 1-1
-------------------------------------------------------------------------
Clock Hour | 8
Minute | 1
(DEPARTING) Station ID: 1 | Level 14
-------------------------------------------------------------------------
(DESTINATION) Station ID: 2 | Level: 35
-------------------------------------------------------------------------
Minute | 2
(CUSTOMER ARRIVAL) Station ID: 1 | Level: 12 | Capacity: 20 | Bike ID 1-3
-------------------------------------------------------------------------
Minute | 3
(CUSTOMER ARRIVAL) Station ID: 1 | Level: 11 | Capacity: 20 | Bike ID 1-4
-------------------------------------------------------------------------
Minute | 4
(DEPARTING) Station ID: 2 | Level 35
-------------------------------------------------------------------------
(DESTINATION) Station ID: 2 | Level: 35
-------------------------------------------------------------------------
Minute | 5
(BIKE ARRIVAL) Station ID: 2 | Level